In [1]:
USE_SAME_FEW_SHOTS=True

In [2]:
number_of_shots = 3

In [26]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
# model_name = "meta-llama/Llama-2-13b-chat-hf"

In [3]:
!pip install -q openai --progress-bar off

In [6]:
!pip install -qlangchain --progress-bar off

  Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
  Using cached jsonpointer-2.4-py2.py3-none-any.whl (7.8 kB)


In [7]:
import pandas as pd

import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

import pickle

import glob

# import openai
# from openai import OpenAI

from typing import Union
from IPython.display import display, HTML

from sklearn.metrics import classification_report

In [8]:
import os
from IPython.display import clear_output

os.environ["OPENAI_API_BASE"] = 'https://api.endpoints.anyscale.com/v1'
os.environ["OPENAI_API_KEY"] = input("INSERT YOUR ANYSCALE API KEY:")

clear_output()

In [9]:
output_datapath = "data/output/2023.11.28-1_Llama2_7B_API_3-few-shot_SEMANTIC_SIMILARITY/"

In [10]:
# # llm = openai.ChatCompletion.create(
# #     model = 'meta-llama/Llama-2-7b-chat-hf', 
# #     messages = [{"role": "system", "content": prompt}],
# #     max_tokens = 150,
# #     response_format = {"type": "json_object"},
# #     stream = True
# # )

# client = OpenAI()

# chat_completion = client.chat.completions.create(
#     messages=[
#         {
#             "role": "user",
#             "content": "Say this is a test",
#         }
#     ],
#     model='meta-llama/Llama-2-7b-chat-hf',
# )

In [11]:
import os
import requests

s = requests.Session()

api_base = os.getenv("OPENAI_API_BASE")
token = os.getenv("OPENAI_API_KEY")
url = f"{api_base}/chat/completions"

In [12]:
# if USE_SAME_FEW_SHOTS==True:
#     data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx").drop(columns=['Unnamed: 0'])
#     data_sel_4_few_shot_examples = data_sel_4_few_shot_examples[
#         (data_sel_4_few_shot_examples.text_id==5614) |
#         (data_sel_4_few_shot_examples.text_id==2089) |
#         (data_sel_4_few_shot_examples.text_id==5665) |
#         (data_sel_4_few_shot_examples.text_id==370)  |
#         (data_sel_4_few_shot_examples.text_id==1422) |
#         (data_sel_4_few_shot_examples.text_id==4329) |
#         (data_sel_4_few_shot_examples.text_id==4929)
#     ]
# else:
#     data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx").drop(columns=['Unnamed: 0']).sample(5)

# data_sel_4_few_shot_examples

In [13]:
top7_train_data_sim_to_each_test_data_filename = "../data/input/top7_train_data_sim_to_each_test_data.pickle"

with open(top7_train_data_sim_to_each_test_data_filename, 'rb') as f:
    top7_train_data_sim_to_each_test_data = pickle.load(f)

In [14]:
# few_shots = ""

# for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
#     input_text = "Input text: \""+row['text']+"\"\n"
#     output_text = "Output: \"---SDG "+str(row['sdg'])+'---\"\n\n'
    
#     few_shots += input_text
#     few_shots += output_text
    
# print(few_shots)

In [15]:
data = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_1020_texts_TEST.xlsx")
data = data.drop(columns=['Unnamed: 0']) #, 'goal_label', 'motivation_for_the_answer', 'goal_label_int', 'sdg_probability'])
data

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0


In [22]:
data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx")

In [16]:
# template_part1 = """[INST]
# <<SYS>>
# In the following lines, a text enclosed in triple quotes will be given to you.

# The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

# The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

# If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "---SDG 0---".

# The following are 7 examples of expected classification given an input text:

# """+few_shots+"""

# The text to be processed is:
# <</SYS>>

# '''
# """

# template_part2 = """
# '''

# <<SYS>>
# Answer with label only usign the format:---LABEL---.
# <</SYS>>
# [/INST]
# """

In [17]:
#data.iloc[181:183]

In [29]:
# data_sel = data #.sample(2)
data_sel = data.iloc[54:] #.sample(2)

In [19]:
# body = {
#   "model": "meta-llama/Llama-2-7b-chat-hf",
#   "messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "Say 'test'."}],
#   "temperature": 0.0
# }

# with s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body) as resp:
#     print(resp.json())

In [30]:
%%time
#results = data_sel.apply(lambda x: llm(prompt.format(text=x["text"])), axis=1)

# cnt = 0
cnt = 54

results = []

for index, row in data_sel.iterrows():

  #print("\n\n##################################################")

  #print("EXPECTED_SDG:", row["sdg"], "TEXT:", row["text"].replace('\n',''),'\n\n')

  few_shots_list = top7_train_data_sim_to_each_test_data[row['text_id']][:number_of_shots]

  few_shots = ""

  for train_data_id in few_shots_list:

    # print("##################################################")
    # print(train_data_id)

    train_row = data_sel_4_few_shot_examples[data_sel_4_few_shot_examples.text_id==train_data_id]

    # print(train_row)

    # print(train_row.text.values[0])
    # print(train_row.sdg.values[0])

    input_text = "Input text: \""+train_row.text.values[0].replace('\n','')+"\"\n"
    output_text = "Output: \"---SDG "+str(train_row.sdg.values[0])+'---\"\n\n'

    few_shots += input_text
    few_shots += output_text

  #print(few_shots)

  template = """[INST]
<<SYS>>
Classify the following text between triple quotes according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015, choosing one (and only one) of the SDG labels based on it best representing the content of the entire text enclosed in triple quotes.

In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

The following are """+str(number_of_shots)+""" non exhaustive examples of expected classification given an input text:

"""+few_shots+"""


The text to be processed is:
<</SYS>>

'''
"""+row["text"]+"""
'''

<<SYS>>
Answer with label only usign the format:---LABEL---.
<</SYS>>
[/INST]
"""



  # print(template)

  text_id = row['text_id']
  text = row["text"]
  sdg_true = row["sdg"]

  #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
  query = template

  body = {
    "model": model_name,
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."}, 
        {"role": "user", "content": query}],
    "temperature": 0.0
  }


  try:
      result = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body).json()
      print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
  except Exception as e:
#       print("#############################################################################")
#       print(e, result)
      print(cnt, text_id, sdg_true, "ERROR:"+str(e)+str(result))
#       print("#############################################################################")
  #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
    
  result_extended = (cnt, text_id, sdg_true, result)

  #print(result_extended)

  with open(output_datapath+f'/sdg_classification_Llama2_7B_SEMANTIC_SIMILARITY_{cnt}.pkl', 'wb') as f:
      pickle.dump(result_extended, f)

  results.append(result_extended)

  cnt+=1

  # if cnt==5:
  #   break



##################################################
EXPECTED_SDG: 0 TEXT: Swiss bank Sygnum has opened a crypto brokerage to satisfy rising institutional demand benefiting crypto in developing markets.The company recently secured a license from the Monetary Authority of Singapore to offer institutional trading and custody of crypto assets.Sygnum Rides Institutional Demand Created by US Bank FailuresThe firm noticed a spike in interest in institutional demand for brokerage services after the failures of US banks Silvergate Capital and Signature Bank.Deposits of Swiss francs reached a record $3.3 billion at the end of last month, Sygnum’s Singapore CEO Gerald Goh said.Funding for Crypto Companies Remained Healthy in 2022 | Source: StatistaSponsored SponsoredThe company will offer brokerage and custody services for digital assets and will match buy and sell orders through its counterparty Sygnum Bank in Switzerland.According to Goh, the crypto brokerage will not handle fiat on and off-ra

54 1497 0 ERROR:'choices'{'generated_text': None, 'embedding_outputs': None, 'num_input_tokens': None, 'num_input_tokens_batch': None, 'num_generated_tokens': None, 'num_generated_tokens_batch': None, 'preprocessing_time': None, 'generation_time': None, 'timestamp': 1701191671.7637122, 'finish_reason': None, 'error': {'message': 'rayllm.backend.llm.error_handling.PromptTooLongError: Input too long. Recieved 4269 tokens, but the maximum input length is 4096 tokens. (Request ID: 030fff228ebb51399dd7773f44be9532)', 'internal_message': 'rayllm.backend.server.openai_compat.openai_exception.OpenAIHTTPException (Request ID: 030fff228ebb51399dd7773f44be9532)', 'code': 400, 'type': 'OpenAIHTTPException', 'param': {}}, 'num_total_tokens': 0, 'num_total_tokens_batch': 0, 'total_time': None}


##################################################
EXPECTED_SDG: 0 TEXT: Cochlea cell atlas built from single-cell sequencing discovers new cell types, uncovers hidden molecular features  Phys.org 


Input t

56 2463 0   ---SDG 0---


##################################################
EXPECTED_SDG: 0 TEXT: Final Ariane 5 launch scheduled for July 4 after fixes to booster separation system – Spaceflight Now  Spaceflight NowAriane 6 | Mobile gantry removal test timelapse  European Space Agency, ESAAriane 6 progress toward inaugural flight: ArianeGroup, Les Mureaux, France  European Space AgencyView Full Coverage on Google News 


Input text: "Tour de France 2023: How to watch, dates, times livestream info and more  Yahoo SportsView Full Coverage on Google News"
Output: "---SDG 0---"

Input text: "Drake reschedules July 1 concert in Columbus  NBC4 WCMH-TVView Full Coverage on Google News"
Output: "---SDG 0---"

Input text: "Titan sub implosion: US Coast guard launches investigation into disaster  BBCCanada begins Titan safety probe  ReutersVoice recordings and data from the mother ship that carried the Titan submersible before it imploded will be examined, investigators say  CNNInternational g

60 6573 1   ---SDG 1---


##################################################
EXPECTED_SDG: 1 TEXT: The indigence rate in Panama increased by 1.5 percentage points, while the other countries with information available (the Dominican Republic, El Salvador and Paraguay) did not register significant changes in their poverty and indigence rates (see figure 1.3). Panama is an exception, as despite the absence of significant changes in the poverty rate it registers an appreciable rise in the poverty gap squared index, revealing a worsening of the situation among the poorest. The poverty gap squared index also rose by slightly more than the poverty' rate in Mexico. 


Input text: "In Honduras and Mexico, the poverty rate rose by between 2% and 3% a year (see table I.2). The poverty gap indicator weights the percentage of poor by the average gap between their incomes and the poverty line, thus, it considers how poor the poor are. The squared poverty gap or poverty severity index shows something

63 1446 1   ---SDG 1---


##################################################
EXPECTED_SDG: 1 TEXT: Including socio-economic indicators related to food security, robustness and diversification of livelihoods, access to quality housing, water and sanitation, savings, school attendance for children, etc. For example, international economists criticised evidence of the economic growth impact of South Africa’s system of social cash transfers at a conference organised by the government of South Africa’s Committee of Inquiry for Comprehensive Social Security held in Cape Town in 2000 (“Towards a Sustainable and Comprehensive Social Security System”). Specifically, the evidence that cash transfers had a greater impact on reducing liquidity constraints to labour market participation and strengthening risk management barriers to investment in job search contradicted the conventional wisdom that reducing the personal costs of unemployment would undermine incentives to work and create dependency. 

...

1007 1794 16   ---SDG 16---


##################################################
EXPECTED_SDG: 16 TEXT: Since the beginning of the Nuremberg trial, the status of the individual in international law has changed. This change is intimately connected with the rights of defence in criminal proceedings, especially in the context of international criminal proceedings. Today, as a matter of right, the individual may make certain claims in international law, and especially in international criminal law and international human rights law related to criminal procedure and substantive criminal law, without relying on a State to make them on his or her behalf. This brief article explores this development of the international legal personality of individuals. It surveys the sources and evidence of international law that can be used by individual defendants in international criminal cases. Finally, it considers an important limit of this development, concerning whether acts of individuals and other p

1010 535 16   ---SDG 16---


##################################################
EXPECTED_SDG: 16 TEXT: Purpose – This article aims to explore the concept of achieving the “right to the city” for marginalised communities. It uses human rights instruments and regeneration best practice to develop a toolkit of indicators for urban regeneration. The article contributes to the literature on realising economic, social and cultural rights encompassed in the “right to the city”.Design/methodology/approach – The article adopts an interdisciplinary approach, involving human rights law, urban planning, housing studies, community development, housing law and social policy. It draws on primary qualitative (participative and observatory) research undertaken by the author while implementing a human rights based approach in an Irish inner‐city local authority estate from 2009 to 2013.Findings – The human rights framework can be adapted to develop a set of measurable regeneration indicators. This artic

1013 6330 16   ---SDG 16---


##################################################
EXPECTED_SDG: 16 TEXT: This article examines the neo-conservative critique of global liberal governance. It provides a theoretically oriented assessment of the neo-conservative case against international law and human rights regimes, and draws out the main political and ethical implications for American democracy and American foreign policy. It is argued that the neo-conservative critique of global governance rests upon an interpretation of the normative order that weaves together democracy, individual rights and national autonomy through a volatile identity politics which is fundamentally at odds with both the pluralist character of ‘Westphalian diplomacy’ and the universal order of rights envisaged by advocates of global governance. More than just the policy autonomy of the US, what is really at stakes in those debates for neo-conservatives is the whole structure of cultural and socio-economic interests 

1016 6628 16   ---SDG 16---


##################################################
EXPECTED_SDG: 16 TEXT: Accountability of government officials' actions is strongly related to the exercise of government authority. In carrying out duties to realize the general welfare, the authority used by organs or government officials is based on the provisions of the laws and regulations (the principle of legality). However, it is not uncommon for the task to be carried out based on discretionary authority. The freedom of government officials to make decisions based on discretionary authority has a great potential to be abused which results in consequences from both point of view of administrative law as well as of criminal law. In the practice, there is discrepancy among law enforcers on the understanding of the principles related to the accountability of discretionary authority held by government officials. 


Input text: "The aim of this essay is to identify a legal basis for accountability obliga

1019 1947 16   ---SDG 16---
CPU times: user 5.21 s, sys: 512 ms, total: 5.72 s
Wall time: 9min 26s


In [ ]:
# import time

# cnt = 0
# # cnt = 181

# results = []
# for index, row in data_sel.iterrows():
    
#     text_id = row['text_id']
#     text = row["text"]
#     sdg_true = row["sdg"]
    
#     query = template_part1 + text + template_part2
    
#     body = {
#       "model": "meta-llama/Llama-2-13b-chat-hf",
#       "messages": [
#           {"role": "system", "content": "You are a helpful assistant."}, 
#           {"role": "user", "content": query}],
#       "temperature": 0.0
#     }
    
        
#     try:
#         result = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body).json()
#         print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
#     except Exception as e:
#         print("#############################################################################")
#         print(e)
#         print("#############################################################################")
        
    
#     result_extended = (cnt, text_id, sdg_true, result)
    
#     with open(output_datapath+f'sdg_classification_Llama2_13B_{cnt}.pkl', 'wb') as f:
#         pickle.dump(result_extended, f)
    
#     results.append(result_extended)
#     cnt+=1
#     time.sleep(0.5)

In [31]:
files = glob.glob(output_datapath + "*.pkl")
# files

In [32]:
# df_list = (pd.DataFrame(pd.read_pickle(file), columns=['cnt', 'id', 'true_label', 'predicted_label']) for file in files)
# df_list = (pd.DataFrame(pd.read_pickle(file)) for file in files)

tuple_list = []
for file in files:
    res = pd.read_pickle(file)
    try:
        if len(res[3]['choices'])>1:
            raise Exception('ERROR: Multilabel detected!')
            break
        sdg_pred = res[3]['choices'][0]['message']['content']
        res4df = (res[0], res[1], res[2], sdg_pred)
        print(res4df)
        tuple_list.append(res4df)
    except: 
        print("#################################################################")
        print(res)
        print("#################################################################")
        

(289, 1960, 4, '  ---SDG 4---')
(783, 2676, 13, '  ---SDG 17---')
(246, 3956, 4, '  ---SDG 4---')
(50, 2541, 0, '  ---SDG 12---')
(288, 4205, 4, '  ---SDG 4---')
(27, 940, 0, '  ---SDG 0---')
(109, 2652, 1, '  ---SDG 8---')
(698, 3494, 11, '  ---SDG 9---')
(156, 5694, 2, '  ---SDG 2---')
(133, 98, 2, '  ---SDG 2---')
(22, 6576, 0, '  ---SDG 9---')
(452, 4797, 7, '  ---SDG 7---')
(378, 3247, 6, '  ---SDG 6---')
(503, 4654, 8, '  ---SDG 8---')
(396, 3056, 6, '  ---SDG 6---')
(856, 843, 14, '  ---SDG 15---')
(475, 3074, 7, '  ---SDG 7---')
(51, 6224, 0, '  ---SDG 5---')
(970, 3501, 16, '  ---SDG 16---')
(956, 2906, 15, ' ---SDG 15---')
(731, 862, 12, '  ---SDG 12---')
(179, 331, 2, '  ---SDG 2---')
(534, 3772, 8, '  ---SDG 8---')
(751, 1230, 12, '  ---SDG 12---')
(789, 611, 13, '  ---SDG 13---')
(398, 6331, 6, '  ---SDG 6---')
(582, 714, 9, '  ---SDG 9---')
(561, 2930, 9, '  ---SDG 9---')
(757, 2238, 12, '  ---SDG 12---')
(845, 4721, 14, '  ---SDG 14---')
(725, 5328, 12, '  ---SDG 12---')

In [33]:
len(tuple_list)

1019

In [34]:
results_processed = pd.DataFrame(tuple_list, columns=['cnt', 'id', 'true_label', 'predicted_label'])

In [35]:
results_processed

,cnt,id,true_label,predicted_label
0,289,1960,4,---SDG 4---
1,783,2676,13,---SDG 17---
2,246,3956,4,---SDG 4---
3,50,2541,0,---SDG 12---
4,288,4205,4,---SDG 4---
...,...,...,...,...
1014,691,5312,11,---SDG 11---
1015,137,2747,2,---SDG 12---
1016,513,4489,8,---SDG 8---
1017,723,2110,12,---SDG 12---


In [36]:
results_processed_merged = data.merge(results_processed, right_on='id', left_on='text_id')
results_processed_merged

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,6668,0,---SDG 3---
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,6099,0,---SDG 8---
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,6471,0,---SDG 16---
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,1786,0,---SDG 4---
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,2143,0,---SDG 17---
...,...,...,...,...,...,...,...,...,...,...,...
1014,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,1015,4695,16,---SDG 16---
1015,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,1016,6628,16,---SDG 16---
1016,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,1017,4563,16,---SDG 16---
1017,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,1018,6710,16,---SDG 16---


In [37]:
results_processed_merged['predicted_label_clean'] = results_processed_merged['predicted_label'].str.extract(r'SDG (\d+)')
results_processed_merged

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label,predicted_label_clean
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,6668,0,---SDG 3---,3
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,6099,0,---SDG 8---,8
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,6471,0,---SDG 16---,16
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,1786,0,---SDG 4---,4
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,2143,0,---SDG 17---,17
...,...,...,...,...,...,...,...,...,...,...,...,...
1014,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,1015,4695,16,---SDG 16---,16
1015,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,1016,6628,16,---SDG 16---,16
1016,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,1017,4563,16,---SDG 16---,16
1017,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,1018,6710,16,---SDG 16---,16


In [38]:
results_processed_merged.groupby("predicted_label_clean").count()

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label
predicted_label_clean,,,,,,,,,,,
0,23,1,23,23,1,1,1,23,23,23,23
1,61,60,61,61,60,60,60,61,61,61,61
10,55,55,55,55,55,55,55,55,55,55,55
11,46,44,46,46,44,44,44,46,46,46,46
12,83,79,83,83,79,79,79,83,83,83,83
13,46,46,46,46,46,46,46,46,46,46,46
14,49,49,49,49,49,49,49,49,49,49,49
15,64,63,64,64,63,63,63,64,64,64,64
16,74,72,74,74,72,72,72,74,74,74,74


In [39]:
results_processed_merged.to_excel(output_datapath+'report_sdg_classification_Llama2_7B.xlsx')

In [40]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1019
1019
              precision    recall  f1-score   support

           0       0.96      0.37      0.54        59
           1       0.79      0.80      0.79        60
           2       1.00      0.53      0.70        60
           3       0.65      0.93      0.77        60
           4       0.83      0.82      0.82        60
           5       0.89      0.68      0.77        60
           6       0.88      0.93      0.90        60
           7       0.82      0.85      0.84        60
           8       0.52      0.68      0.59        60
           9       0.65      0.85      0.74        60
          10       0.65      0.60      0.63        60
          11       0.83      0.63      0.72        60
          12       0.63      0.87      0.73        60
          13       0.93      0.72      0.81        60
          14       0.98      0.80      0.88        60
          15       0.88      0.93      0.90        60
          16       0.74      0.92      0.82        60
          17     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
